# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer_shift-jis.csv", encoding="shift-jis")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

### はじめに：パイプ演算子について  
パイプ演算子 %>% はRの標準演算子ではなく、dplyrをインストールしたら使えるようになります。  
https://qiita.com/zakkiiii/items/ecb7ddf55c7f1bfacecf  

R-4.1.0 からはパイプ演算子がRの標準装備になるようです。  
The native pipe  
When the {magrittr} package introduced the pipe to R in 2014, it was building upon similar syntax in Unix / linux scripting languages (bash) and functional programming languages (like F#).  
https://www.r-bloggers.com/2021/05/new-features-in-r-4-1-0/

---
> R-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [8]:
df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount=sum(amount)) %>%
    mutate(lag_ymd = lag(sales_ymd), lag_amount = lag(sum_amount), diff_amount = sum_amount - lag_amount) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


sales_ymd,sum_amount,lag_ymd,lag_amount,diff_amount
20170101,33723,NA,NA,NA
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981
20170110,20143,20170109,26718,-6575


In [9]:
head(df_receipt)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138


In [10]:
dim(df_receipt)

[1] 104681      9

In [11]:
# パイプ演算子を使わずにステップワイズに実行
# https://kazutan.github.io/kazutanR/hands_on_170730/group_by.html
# groupe_by()をしても見た目は変わらないが、まとめて処理ができるようになっている
df_tmp <- group_by(df_receipt, sales_ymd)
head(df_tmp)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138


In [12]:
dim(df_tmp)

[1] 104681      9

In [13]:
df_tmp2 <- summarise(df_tmp, sum_amount=sum(amount))
head(df_tmp2)

`summarise()` ungrouping output (override with `.groups` argument)


sales_ymd,sum_amount
20170101,33723
20170102,24165
20170103,27503
20170104,36165
20170105,37830
20170106,32387


In [14]:
dim(df_tmp2)

[1] 1034    2

In [15]:
# では groupe_by()をせずに、いきなり summarise() を df_reciept にするとどうなるか？
df_exp <- summarise(df_receipt, sum_amount=sum(amount))
head(df_exp)

sum_amount
33556550


In [16]:
# mutate() について：https://kazutan.github.io/kazutanR/hands_on_170730/mutate.html
# 新しい列を既存の列から作る
# lag() について：https://kazutan.github.io/kazutanR/hands_on_170730/mutate.html
df_tmp3 <- mutate(df_tmp2, lag_ymd = lag(sales_ymd), lag_amount = lag(sum_amount), diff_amount = sum_amount - lag_amount)
head(df_tmp3)

sales_ymd,sum_amount,lag_ymd,lag_amount,diff_amount
20170101,33723,NA,NA,NA
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443


In [17]:
# lag() でN個ずらすなら lag(元のカラム名, n=N)
df_tmp4 <- mutate(df_tmp2, lag_ymd = lag(sales_ymd, n=2), lag_amount = lag(sum_amount, n=3))
head(df_tmp4)

sales_ymd,sum_amount,lag_ymd,lag_amount
20170101,33723,NA,NA
20170102,24165,NA,NA
20170103,27503,20170101,NA
20170104,36165,20170102,33723
20170105,37830,20170103,24165
20170106,32387,20170104,27503


---
> R-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [18]:
# 模範解答をそのまま載せます。一行ずつ丁寧に解釈していけば理解はできますが、なかなか自分でこのコードは書けませんね。
# 縦持ちケースなら慣れたらできるかも知れませんが、横持ちケースは難しい！

In [19]:
# コード例1:縦持ちケース
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount=sum(amount))

for (i in 1:3){
    if (i == 1){
        df_lag <- df_sum %>%
            mutate(lag_ymd = lag(sales_ymd, n=i), lag_amount = lag(sum_amount, n=i))
    }
    else{
        df_tmp <- df_sum %>%
            mutate(lag_ymd = lag(sales_ymd, n=i), lag_amount = lag(sum_amount, n=i))
        df_lag <- rbind(df_lag, df_tmp)
    }
}

drop_na(df_lag, everything()) %>%
    arrange(sales_ymd, lag_ymd) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


sales_ymd,sum_amount,lag_ymd,lag_amount
20170102,24165,20170101,33723
20170103,27503,20170101,33723
20170103,27503,20170102,24165
20170104,36165,20170101,33723
20170104,36165,20170102,24165
20170104,36165,20170103,27503
20170105,37830,20170102,24165
20170105,37830,20170103,27503
20170105,37830,20170104,36165
20170106,32387,20170103,27503


In [20]:
# コード例2:横持ちケース
df_sum <-df_receipt %>%
    group_by(sales_ymd) %>%
    summarise(sum_amount=sum(amount))

for (i in 1:3){
    if (i == 1){
        df_lag <- df_sum %>%
            mutate(lag_ymd_1 = lag(sales_ymd, n=i), lag_amount_1 = lag(sum_amount, n=i))
    }
    else{
        col_name_1 <- paste("lag_ymd", i , sep="_")
        col_name_2 <- paste("lag_amount", i , sep="_")
        print(col_name_1)
        print(col_name_2)
        df_tmp <- df_sum %>%
            mutate(!!col_name_1 := lag(sales_ymd, n=i), !!col_name_2 := lag(sum_amount, n=i))
        df_lag <- cbind(df_lag, df_tmp[c(col_name_1, col_name_2)])
    }
}

drop_na(df_lag, everything()) %>%
    arrange(sales_ymd) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


[1] "lag_ymd_2"
[1] "lag_amount_2"
[1] "lag_ymd_3"
[1] "lag_amount_3"


sales_ymd,sum_amount,lag_ymd_1,lag_amount_1,lag_ymd_2,lag_amount_2,lag_ymd_3,lag_amount_3
20170104,36165,20170103,27503,20170102,24165,20170101,33723
20170105,37830,20170104,36165,20170103,27503,20170102,24165
20170106,32387,20170105,37830,20170104,36165,20170103,27503
20170107,23415,20170106,32387,20170105,37830,20170104,36165
20170108,24737,20170107,23415,20170106,32387,20170105,37830
20170109,26718,20170108,24737,20170107,23415,20170106,32387
20170110,20143,20170109,26718,20170108,24737,20170107,23415
20170111,24287,20170110,20143,20170109,26718,20170108,24737
20170112,23526,20170111,24287,20170110,20143,20170109,26718
20170113,28004,20170112,23526,20170111,24287,20170110,20143


---
> R-043: レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [21]:
# df_customer が一部データがずれていることを発見。
# まずその対応
# df_customer <- read.csv("./data/customer_shift-jis.csv", encoding="shift-jis")

In [22]:
df_sales_summary <- df_customer[c("customer_id", "gender_cd", "birth_day" , "age")] %>%
    mutate(era = trunc(age / 10) * 10) %>%
    inner_join(df_receipt, by="customer_id") %>%
    group_by(gender_cd, era) %>%
    summarise(sum_amount=sum(amount)) %>%
    spread(gender_cd, sum_amount, fill=0) %>%
    rename(male='0', female='1', unknown='9')

df_sales_summary

`summarise()` regrouping output by 'gender_cd' (override with `.groups` argument)


era,male,female,unknown
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418
70,13435,29764,2427
80,46360,262923,5111
90,0,6260,0


In [23]:
# 一行ずつパイプ演算子を使わずに解釈しながら実行
head(df_customer)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0


In [24]:
dim(df_customer)

[1] 21971    11

In [25]:
df_tmp <- df_customer[c("customer_id", "gender_cd", "birth_day" , "age")]
head(df_tmp)

customer_id,gender_cd,birth_day,age
CS021313000114,1,1981-04-29,37
CS037613000071,9,1952-04-01,66
CS031415000172,1,1976-10-04,42
CS028811000001,1,1933-03-27,86
CS001215000145,1,1995-03-29,24
CS020401000016,0,1974-09-15,44


In [26]:
dim(df_tmp)

[1] 21971     4

In [27]:
df_tmp2 <- mutate(df_tmp, era = trunc(age / 10) * 10) 
head(df_tmp2)

customer_id,gender_cd,birth_day,age,era
CS021313000114,1,1981-04-29,37,30
CS037613000071,9,1952-04-01,66,60
CS031415000172,1,1976-10-04,42,40
CS028811000001,1,1933-03-27,86,80
CS001215000145,1,1995-03-29,24,20
CS020401000016,0,1974-09-15,44,40


In [28]:
dim(df_tmp2)

[1] 21971     5

In [29]:
df_tmp3 <- inner_join(df_tmp2, df_receipt, by="customer_id")
head(df_tmp3)

customer_id,gender_cd,birth_day,age,era,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
CS031415000172,1,1976-10-04,42,40,20170507,1210118400,S13031,1102,1,P060103001,1,100
CS031415000172,1,1976-10-04,42,40,20171026,1224979200,S13031,1182,1,P090203004,1,320
CS031415000172,1,1976-10-04,42,40,20190325,1269475200,S13031,1192,1,P071401025,1,2400
CS031415000172,1,1976-10-04,42,40,20170111,1200009600,S13031,1132,2,P071203007,1,448
CS031415000172,1,1976-10-04,42,40,20190325,1269475200,S13031,1192,2,P070805011,1,258
CS031415000172,1,1976-10-04,42,40,20190122,1264118400,S13031,1142,2,P050102006,1,215


In [30]:
dim(df_tmp3)

[1] 65682    13

In [31]:
df_tmp4 <- group_by(df_tmp3, gender_cd, era) %>% summarise(sum_amount=sum(amount))
head(df_tmp4)

`summarise()` regrouping output by 'gender_cd' (override with `.groups` argument)


gender_cd,era,sum_amount
0,10,1591
0,20,72940
0,30,177322
0,40,19355
0,50,54320
0,60,272469


In [32]:
dim(df_tmp4)

[1] 25  3

In [33]:
# spread(): https://www.bioinfoblog.com/entry/2020/09/20/%E3%80%90R%E3%80%91gather/spread%E3%82%92%E4%BD%BF%E3%81%84%E3%81%93%E3%81%AA%E3%81%99%EF%BC%81
df_tmp5 <- spread(df_tmp4, gender_cd, sum_amount, fill=0)
head(df_tmp5)

era,0,1,9
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418


In [34]:
rename(df_tmp5, male='0', female='1', unknown='9')

era,male,female,unknown
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418
70,13435,29764,2427
80,46360,262923,5111
90,0,6260,0


---
> R-044: 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [35]:
gather(df_sales_summary, key = gender_cd, value = sum_amount, male, female, unknown) %>%
    mutate(gender_cd = case_when(
        gender_cd == "male"  ~ "00",
        gender_cd == "female"  ~ "01",
        gender_cd == "unknown"  ~ "99",
    ))

era,gender_cd,sum_amount
10,00,1591
20,00,72940
30,00,177322
40,00,19355
50,00,54320
60,00,272469
70,00,13435
80,00,46360
90,00,0
10,01,149836


In [36]:
# 分解して解釈
df_tmp <- gather(df_sales_summary, key = gender_cd, value = sum_amount, male, female, unknown)
head(df_tmp, n=10)

era,gender_cd,sum_amount
10,male,1591
20,male,72940
30,male,177322
40,male,19355
50,male,54320
60,male,272469
70,male,13435
80,male,46360
90,male,0
10,female,149836


In [37]:
# mutate() 中での条件文：https://ando-roid.hatenablog.com/entry/2019/04/17/124842
mutate(df_tmp, gender_cd = case_when(
        gender_cd == "male"  ~ "00",
        gender_cd == "female"  ~ "01",
        gender_cd == "unknown"  ~ "99",
    ))

era,gender_cd,sum_amount
10,00,1591
20,00,72940
30,00,177322
40,00,19355
50,00,54320
60,00,272469
70,00,13435
80,00,46360
90,00,0
10,01,149836


---
> R-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [38]:
# なぜエラーになるのか？
df_tmp <- cbind(df_customer['customer_id'], strftime(df_customer$birth_day, format="%Y%m%d"))

ERROR: Error in as.POSIXlt.character(as.character(x), ...): character string is not in a standard unambiguous format


In [39]:
df_customer$birth_day[1]

[1] 1981-04-29
12759 Levels: 1928-11-26 1928-12-14 1929-01-07 1929-01-12 ... 2007-11-25

In [40]:
strftime(df_customer$birth_day[100], format="%Y%m%d")

[1] "19641208"

In [41]:
# 個別の行ではできるのに、一度にはできない
strftime(df_customer$birth_day, format="%Y%m%d")

ERROR: Error in as.POSIXlt.character(as.character(x), ...): character string is not in a standard unambiguous format


In [42]:
# forループでベクトルに要素を追加してみる
# for ループの実験
for (i in 1:10){
    print(strftime(df_customer$birth_day[i], format="%Y%m%d"))}

[1] "19810429"
[1] "19520401"
[1] "19761004"
[1] "19330327"
[1] "19950329"
[1] "19740915"
[1] "19770809"
[1] "19730817"
[1] "19310502"
[1] "19620711"


In [43]:
# 行数取得
rn = nrow(df_customer)
rn

[1] 21971

In [48]:
print(c(strftime(df_customer$birth_day[8], format="%Y%m%d"), strftime(df_customer$birth_day[9], format="%Y%m%d")))

[1] "19730817" "19310502"


In [54]:
# forループでベクトルに要素を追加してみる
# ベクトルに要素追加：http://cse.naro.affrc.go.jp/takezawa/r-tips/r/15.html
# 先ず10行で簡単な例で実験
vec = c()
for (i in 1:10){
    print(i)
    vec <- append(vec, i)
}
vec

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10


[1]  1  2  3  4  5  6  7  8  9 10

In [55]:
vec = c()
for (i in 1:10){
    print(i)
    vec <- append(vec, i)
}
vec

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10


[1]  1  2  3  4  5  6  7  8  9 10

In [56]:
# 10行で実験
birth_day = c()
for (i in 1:10){
    birth_day <- append(birth_day, strftime(df_customer$birth_day[i], format="%Y%m%d"))}
birth_day

[1] "19810429" "19520401" "19761004" "19330327" "19950329" "19740915"
 [7] "19770809" "19730817" "19310502" "19620711"

In [59]:
# 本番 やはり同じエラー
birth_day = c()
for (i in 1:rn){
    birth_day <- append(birth_day, strftime(df_customer$birth_day[i], format="%Y%m%d"))}
birth_day

ERROR: Error in as.POSIXlt.character(as.character(x), ...): character string is not in a standard unambiguous format


In [60]:
# 100行で実験
birth_day = c()
for (i in 1:100){
    birth_day <- append(birth_day, strftime(df_customer$birth_day[i], format="%Y%m%d"))}
birth_day

[1] "19810429" "19520401" "19761004" "19330327" "19950329" "19740915"
  [7] "19770809" "19730817" "19310502" "19620711" "19750818" "19540716"
 [13] "19920201" "19751228" "19770105" "19760117" "19800415" "19390702"
 [19] "19530406" "19741022" "19530830" "19770927" "19830317" "19611217"
 [25] "19600327" "19800409" "19510411" "19621109" "19860326" "19741227"
 [31] "19691003" "19831126" "19700226" "19930928" "19811218" "19300123"
 [37] "19890608" "19411231" "19950119" "19621214" "19551205" "19530615"
 [43] "19700524" "19811005" "19771107" "19850121" "19571226" "19820331"
 [49] "19710510" "19710314" "19701130" "19501112" "19400208" "19770627"
 [55] "19690927" "19840211" "19970203" "19500518" "19510209" "19721004"
 [61] "19690208" "19741126" "19650601" "19430824" "19501105" "19640810"
 [67] "19640625" "19671206" "19620312" "19660717" "19700817" "19601129"
 [73] "19640531" "19620310" "19670102" "19610930" "19300505" "19571016"
 [79] "19590604" "19670727" "19750826" "19860125" "19651224" "19850214"
 [85] "19710919" "19870619" "19760712" "19410703" "19621018" "19920729"
 [91] "19750213" "19600111" "19470714" "19731019" "19790926" "19430511"
 [97] "19690903" "19510820" "19750710" "19641208"

In [ ]:
21971

In [69]:
# 100行でずつ増やして実験
birth_day = c()
for (i in 1:900){
    birth_day <- append(birth_day, strftime(df_customer$birth_day[i], format="%Y%m%d"))}
birth_day

[1] "19810429" "19520401" "19761004" "19330327" "19950329" "19740915"
  [7] "19770809" "19730817" "19310502" "19620711" "19750818" "19540716"
 [13] "19920201" "19751228" "19770105" "19760117" "19800415" "19390702"
 [19] "19530406" "19741022" "19530830" "19770927" "19830317" "19611217"
 [25] "19600327" "19800409" "19510411" "19621109" "19860326" "19741227"
 [31] "19691003" "19831126" "19700226" "19930928" "19811218" "19300123"
 [37] "19890608" "19411231" "19950119" "19621214" "19551205" "19530615"
 [43] "19700524" "19811005" "19771107" "19850121" "19571226" "19820331"
 [49] "19710510" "19710314" "19701130" "19501112" "19400208" "19770627"
 [55] "19690927" "19840211" "19970203" "19500518" "19510209" "19721004"
 [61] "19690208" "19741126" "19650601" "19430824" "19501105" "19640810"
 [67] "19640625" "19671206" "19620312" "19660717" "19700817" "19601129"
 [73] "19640531" "19620310" "19670102" "19610930" "19300505" "19571016"
 [79] "19590604" "19670727" "19750826" "19860125" "19651224" "19850214"
 [85] "19710919" "19870619" "19760712" "19410703" "19621018" "19920729"
 [91] "19750213" "19600111" "19470714" "19731019" "19790926" "19430511"
 [97] "19690903" "19510820" "19750710" "19641208" "19650421" "19441114"
[103] "19800920" "19590204" "19970418" "19800824" "19711030" "19760915"
[109] "19720616" "19450416" "19630423" "19600902" "19830203" "19730915"
[115] "19530425" "19691126" "19720606" "19561023" "19621030" "19731201"
[121] "19740710" "19411121" "19531018" "19521120" "19740225" "19830826"
[127] "19500223" "19290704" "19550307" "19690425" "19490818" "19650617"
[133] "19430725" "19610425" "19531009" "19830119" "19820316" "19641014"
[139] "19630505" "19430805" "19670105" "19670106" "19521112" "19810901"
[145] "19740521" "19600509" "19591126" "19690430" "19810604" "19730428"
[151] "19630903" "19810429" "19820417" "19620427" "19731109" "20061124"
[157] "19691018" "19500923" "19730216" "19590905" "19590813" "19711227"
[163] "19660603" "19780106" "19701125" "19630906" "19820417" "19451129"
[169] "19400225" "19650611" "19711104" "19660217" "19591125" "19700820"
[175] "19720517" "19721121" "19520307" "19961009" "19900725" "19871001"
[181] "19770312" "19731101" "19490427" "19601016" "19660713" "19600103"
[187] "19910525" "19910712" "19671106" "19820211" "19820716" "19870903"
[193] "19730820" "19611018" "19711224" "19801217" "19570902" "19530803"
[199] "19460902" "19660403" "19520914" "19520619" "19300823" "19610916"
[205] "19640113" "19710404" "19700621" "19651017" "19650118" "19731117"
[211] "19621222" "19860323" "19720211" "19651207" "19670607" "19711027"
[217] "19510719" "19730806" "19600206" "19621217" "19791012" "19870512"
[223] "19490811" "19441201" "19720423" "19690531" "19650823" "19600121"
[229] "19410913" "19590805" "19750815" "19900102" "19550525" "19671128"
[235] "19591217" "19770711" "19920528" "19560607" "19701124" "19720622"
[241] "19610901" "19690414" "19600120" "19631209" "19741104" "19801113"
[247] "20021220" "19740422" "19611013" "19631122" "19530219" "19701204"
[253] "19901204" "19681213" "19601119" "19630122" "19860311" "19691024"
[259] "19591127" "19630530" "19621203" "19740318" "19870512" "19820421"
[265] "19520425" "19660102" "19800226" "19620119" "19420628" "19591116"
[271] "19550304" "19700921" "19651120" "19551227" "19840808" "19540615"
[277] "19810204" "19890902" "19730121" "19641207" "19770312" "19660510"
[283] "19730304" "19591217" "19650411" "19590624" "19880108" "19700210"
[289] "19300717" "19501013" "19610621" "19811017" "19610224" "19561029"
[295] "19970626" "19670822" "19610522" "19620904" "19830521" "19800616"
[301] "19410715" "19560428" "19890512" "19700301" "19600117" "19660407"
[307] "19781111" "19720825" "19510609" "19770405" "19851108" "19620820"
[313] "19630625" "19740305" "19490319" "19521216" "19720217" "19630420"
[319] "19750917" "19860305" "19450414" "19740517" "19770228" "19800619"
[325] "19490905" "19650227" "19600520" "19481019" "19500810" "19531214"
[331] "19491026" "19560404" "19740908" "19

In [70]:
# 1000行でずつ増やして実験 ⇒ 失敗 900～1000行の間に問題あり。データがどこかおかしいのか？
birth_day = c()
for (i in 1:1000){
    birth_day <- append(birth_day, strftime(df_customer$birth_day[i], format="%Y%m%d"))}
birth_day

ERROR: Error in as.POSIXlt.character(as.character(x), ...): character string is not in a standard unambiguous format


In [ ]:
# その後の処理は簡単なので、この問題はここまでとする
# コードに誤りはない様子
colnames(df_tmp) <- c("customer_id","birth_day")
head(df_tmp,10)